In [1]:
import shutil
import os
import os.path as op
from collections import OrderedDict

from eptools.people import (get_profiles_registry,
                            fetch_profiles_files,
                            contact_from_dict)

from eptools.badges.factory import (BadgeFactory, 
                                    get_badge_role,
                                    prepare_badge_pdf,
                                    get_badge_template_file)

In [2]:
output_dir = op.expanduser('~/Desktop/eps_badges')
os.makedirs(output_dir, exist_ok=True)

CURRENT_EUROPYTHON_FIRST_TICKET_ID = 3000

factory = BadgeFactory(output_dir)

In [3]:
fetch_data = True

# fetch the data from the DB
if fetch_data:
    profiles_file, talks_file = fetch_profiles_files()
    print(profiles_file, talks_file)

profiles.json accepted_talks.json


In [4]:
# load and prepared the profiles
profiles = get_profiles_registry()

In [5]:
#print('Creating {} badges.'.format(len(profiles.people)))
def create_badge(profile, profiles, factory):
    
    print(profile['id'], end='')

    # read the contact
    contact = contact_from_dict(profile)

    roles   = list(profiles.get_roles_of(contact.email, 
                                         name=contact.name, 
                                         surname=contact.surname))

    # create the SVG file
    svg_file = factory.generate_badge_svg(contact, roles)

    # convert the SVG to PDF
    pdf_file = prepare_badge_pdf(svg_file)

    # make both badge faces and save the result in its place
    outdir = op.join(factory.out_dir, get_badge_role(roles).name)
    os.makedirs(outdir, exist_ok=True)
    
    return outdir, pdf_file


def create_badge(contact, roles, factory):

    # create the SVG file
    svg_file = factory.generate_badge_svg(contact, roles)

    # convert the SVG to PDF
    pdf_file = prepare_badge_pdf(svg_file)

    # make both badge faces and save the result in its place
    outdir = op.join(factory.out_dir, get_badge_role(roles).name)
    os.makedirs(outdir, exist_ok=True)
    
    return outdir, pdf_file

import hashlib

def file_cmp(file1, file2, hash_func=hashlib.sha224):
    os.utime(file1)
    os.utime(file2) # touch
    
    hash1 = hash_func(open(file1, 'rb').read()).hexdigest()
    hash2 = hash_func(open(file2, 'rb').read()).hexdigest()
    return hash1 == hash2

In [6]:
new_badges = []
for profile in profiles:
    if profile.pop('frozen'):
        continue
    if profile['id'] <= CURRENT_EUROPYTHON_FIRST_TICKET_ID:
        continue
    
    contact = contact_from_dict(profile)

    roles   = list(profiles.get_roles_of(contact.email, 
                                         name=contact.name, 
                                         surname=contact.surname))

    outdir, pdf_file = create_badge(contact, roles, factory)
    
    result_file = op.join(outdir, op.basename(pdf_file))

    move = False
    #if op.exists(result_file):
    #    if not file_cmp(pdf_file, result_file): # not equal
    #        move = True
    #else:
    #    move = True
    if not op.exists(result_file):
        move = True

    if move:
        _ = shutil.move(pdf_file, result_file)
        new_badges.append(result_file)
        print(' ✓')
    else:
        print(' X')

3005 X
3009 X
3010 X
3014 X
3021 X
3022 X
3024 X
3029 X
3046 X
3052 X
3059 X
3075 X
3077 X
3080 X
3082 X
3083 X
3085 X
3087 X
3088 X
3089 X
3091 X
3098 X
3099 X
3100 X
3103 X
3104 X
3105 X
3106 X
3107 X
3108 X
3111 X
3120 X
3121 X
3125 X
3127 X
3128 X
3133 X
3134 X
3137 X
3138 X
3142 X
3143 X
3146 X
3148 X
3153 X
3156 X
3157 X
3158 X
3165 X
3166 X
3167 X
3168 X
3169 X
3170 X
3171 X
3176 X
3177 X
3178 X
3179 X
3184 X
3185 X
3186 X
3187 X
3192 X
3193 X
3194 X
3195 X
3196 X
3199 X
3200 X
3201 X
3203 X
3206 X
3207 X
3208 X
3209 X
3212 X
3213 X
3214 X
3221 X
3222 X
3224 X
3231 X
3233 X
3234 X
3235 X
3236 X
3237 X
3238 X
3242 X
3243 X
3244 X
3247 X
3249 X
3250 X
3252 X
3255 X
3265 X
3266 X
3267 ✓
3271 X
3273 X
3278 X
3279 X
3280 X
3281 X
3282 X
3283 X
3285 X
3286 X
3287 X
3288 X
3291 X
3292 X
3293 X
3295 X
3296 X
3297 X
3298 X
3299 X
3301 X
3302 X
3303 X
3305 X
3310 X
3312 X
3313 X
3314 X
3316 X
3322 X
3323 X
3324 X
3325 X
3326 X
3327 X
3331 X
3332 X
3334 X
3337 X
3339 X
3340 X
3341 X
3342 X

In [7]:
import os
import os.path as op
from hansel import Crumb, crumb_copy

outcr = Crumb(op.join(output_dir, 'out',     '{group}', '{badge}'), ignore_list=['.*'])
stocr = Crumb(op.join(output_dir, 'storage', '{group}', '{badge}'), ignore_list=['.*'])

newcr = Crumb(op.join(output_dir, '160714', '{group}', '{badge}'), ignore_list=['.*'])

In [8]:
stocrs = {cr['badge'][0]:str(cr) for cr in stocr.ls('badge')}

diff = []
for ocr in outcr.ls('badge'):
    osiz = os.stat(str(ocr)).st_size
    if ocr['badge'][0] not in stocrs:
        diff.append(ocr)
        continue
    
    stsiz = os.stat(stocrs[ocr['badge'][0]]).st_size
    if osiz != stsiz:
        diff.append(ocr)

In [9]:
for cr in diff:
    crumb_copy(cr, newcr)

Copying /Users/alexandre/Desktop/eps_badges/out/attendee/badge_ep2016_attendee_1968-joined.pdf -> /Users/alexandre/Desktop/eps_badges/160714/attendee/badge_ep2016_attendee_1968-joined.pdf
Copying /Users/alexandre/Desktop/eps_badges/out/attendee/badge_ep2016_attendee_1970-joined.pdf -> /Users/alexandre/Desktop/eps_badges/160714/attendee/badge_ep2016_attendee_1970-joined.pdf
Copying /Users/alexandre/Desktop/eps_badges/out/attendee/badge_ep2016_attendee_3267-joined.pdf -> /Users/alexandre/Desktop/eps_badges/160714/attendee/badge_ep2016_attendee_3267-joined.pdf
Copying /Users/alexandre/Desktop/eps_badges/out/attendee/badge_ep2016_attendee_3486-joined.pdf -> /Users/alexandre/Desktop/eps_badges/160714/attendee/badge_ep2016_attendee_3486-joined.pdf
Copying /Users/alexandre/Desktop/eps_badges/out/attendee/badge_ep2016_attendee_3487-joined.pdf -> /Users/alexandre/Desktop/eps_badges/160714/attendee/badge_ep2016_attendee_3487-joined.pdf
Copying /Users/alexandre/Desktop/eps_badges/out/attendee/bad